## Collecting data for violin plots (SF5 G,H and SF6 G,H)

In [ ]:
import os
import re
import sys
import argparse
import subprocess
import matplotlib
import time
import pandas as pd
import numpy as np
from itertools import groupby
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from pylab import *
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle
import csv
from collections import defaultdict
import operator
import joblib
from sklearn.ensemble import BaggingClassifier
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from Bio import SeqIO
from sklearn.feature_selection import RFE
from sklearn import preprocessing
import pybedtools as pbt
import pyBigWig as pbw
import glob
from datetime import date
import random
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from itertools import chain
import string
import shlex
import shutil
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from sklearn.ensemble import AdaBoostClassifier
from adjustText import adjust_text
import matplotlib.ticker as ticker

TFs_CHS_AFS = pd.read_csv("...", sep="\t", header=None)
files_pwm_HUMAN_mono = TFs_CHS_AFS[0]

In [ ]:
pos_class = 10000
neg_class = pos_class * 100
N_features = 1000         
n_jobs = 100
plot_slim = False
scale_data = True
print_GC = False
verbose = True
organism = "HUMAN_GHTS_CHS"
mode  = "mono"
model_name  = "RandomForestClassifier"
sys.setrecursionlimit = 10**3 # for deeper recursion

#today = date.today()
#today_date = today.strftime("%d.%m.%Y")
today_date = "23.12.2023"

root = "..."
os.chdir(root)
model_dir = os.path.abspath('...')
os.chdir(model_dir)

# H - training set data type
# M - testing set data type

colnames_l = ["Count", "roc_auc_train_H_PWM", "roc_auc_train_H_PWM_di", "roc_auc_test_H_PWM", 
 "roc_auc_test_H_PWM_di", "roc_auc_test_M_PWM", "roc_auc_test_M_PWM_di",
"roc_auc_train_H", "mean_auc_train_H", "median_auc_train_H", "std_auc_train_H",
"roc_auc_test_H", "mean_auc_test_H", "median_auc_test_H", "std_auc_test_H",
"roc_auc_test_M", "mean_auc_test_M", "median_auc_test_M", "std_auc_test_M",
"pr_auc_train_H", "mean_pr_auc_train_H", "median_pr_auc_train_H", "std_pr_auc_train_H",
"pr_auc_test_H", "mean_pr_auc_test_H", "median_pr_auc_test_H", "std_pr_auc_test_H",
"pr_auc_test_M", "mean_pr_auc_test_M", "median_pr_auc_test_M", "std_pr_auc_test_M",
"pr_auc_train_H_PWM", "pr_auc_train_H_PWM_di", "pr_auc_test_H_PWM", "pr_auc_test_H_PWM_di", 
"pr_auc_test_M_PWM", "pr_auc_test_M_PWM_di"]

df_l = []
TF_сalc = 0
for TF in files_pwm_HUMAN_mono:   
    #print(" ")
    #print(f"{TF} #" + " {сalc} of {all_c}".format(all_c=len(files_pwm_HUMAN_mono), сalc=TF_сalc))
    TF_сalc += 1
    ddf = pd.read_csv(f"{TF}_new_log_roc_pr_{mode}_{today_date}.txt",  sep=' ', header=None)
    ddf.columns = colnames_l
    ddf["TF_name"] = TF
    ddf["Model"] = model_name
    ddf["PWM"] = mode
    df_l.append(ddf)

## For training on X and testing on X (the same dataset source)

In [ ]:
df_total = pd.concat(df_l)
df_total = df_total[(df_total["Model"] == "RandomForestClassifier")]
df_total["PR_delta_H"] = df_total["pr_auc_test_H"]-df_total["pr_auc_test_H_PWM"]
df_total["ROC_delta_H"] = df_total["roc_auc_test_H"]-df_total["roc_auc_test_H_PWM"]
df_total = df_total[df_total["PWM"] == "mono"]

In [ ]:
TF_bottom_list = []
TF_top_list = []

model = "RandomForestClassifier"
сalc = 0

df_all = pd.DataFrame(data={x:[] for x in df_total.columns})

for TF in list(set(df_total.TF_name)): 
    print(TF)
    сalc += 1
    sns.set_context("paper", font_scale=3)  
    df = df_total[df_total.TF_name == TF]

    if float(df.tail(1)["roc_auc_test_H"]) - float(df.head(1)["roc_auc_test_H"]) < 0:
        TF_bottom_list.append(TF)
    if float(df.tail(1)["roc_auc_test_H"]) - float(df.head(1)["roc_auc_test_H"]) > 0.05:
        TF_top_list.append(TF)
        
    df["ROC_delta_H"] = df["roc_auc_test_H"] - float(df.tail(1)["roc_auc_test_H_PWM"])
    df['Count'].iloc[-1] = "all"
    df_all = pd.concat([df_all, df])
                   
df_all['PWM'] = df_all['Count']
df_all['PWM'] = df_all['PWM'].astype(str)
df_all["dLR"] = df_all["ROC_delta_H"]

In [ ]:
df_all

In [ ]:
df_all.to_csv(f'../df_violin_..._XX.csv', index=False)

## For training on X and testing on Y

In [ ]:
df_total = pd.concat(df_l)
df_total = df_total[(df_total["Model"] == "RandomForestClassifier")]
df_total["PR_delta_M"] = df_total["pr_auc_test_M"]-df_total["pr_auc_test_M_PWM"]
df_total["ROC_delta_M"] = df_total["roc_auc_test_M"]-df_total["roc_auc_test_M_PWM"]
df_total = df_total[df_total["PWM"] == "mono"]

In [ ]:
TF_bottom_list = []
TF_top_list = []

model = "RandomForestClassifier"
сalc = 0

df_all = pd.DataFrame(data={x:[] for x in df_total.columns})

for TF in list(set(df_total.TF_name)): 
    print(TF)
    сalc += 1
    sns.set_context("paper", font_scale=3)  
    df = df_total[df_total.TF_name == TF]

    if float(df.tail(1)["roc_auc_test_M"]) - float(df.head(1)["roc_auc_test_M"]) < 0:
        TF_bottom_list.append(TF)
    if float(df.tail(1)["roc_auc_test_M"]) - float(df.head(1)["roc_auc_test_M"]) > 0.05:
        TF_top_list.append(TF)
        
    df["ROC_delta_M"] = df["roc_auc_test_M"] - float(df.tail(1)["roc_auc_test_M_PWM"])
    df['Count'].iloc[-1] = "all"
    df_all = pd.concat([df_all, df])
                   
df_all['PWM'] = df_all['Count']
df_all['PWM'] = df_all['PWM'].astype(str)
df_all["dLR"] = df_all["ROC_delta_M"]

In [ ]:
df_all

In [ ]:
df_all.to_csv(f'../df_violin_..._XY.csv', index=False)